In [1]:
import gadata_funcs,pdb,re
import pandas as pd
import numpy as np
import pylab as pl
from gadata_funcs import date2days,gadata_date2days,salesdata_date2days,getFeatures
from importlib import reload
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sys import stdout
from xgboost import XGBClassifier

%matplotlib notebook

In [2]:
# The ranges for the feature generate and target windows respectively
featLen=30
labelLen=20
featThreshold1=50
featThreshold2=100
featThreshold3=150

In [3]:
# Just the relevant features, and formatting for easier use
xdf,ydf=getFeatures()
x=xdf[['hits','maxPurchase']].values
y=ydf.values.reshape((-1,))

/home/wlwoon/Dropbox/chicken_work/impersuasion_stuff/GAdata/gadata_funcs.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  featdf.replace(np.nan,nanReplace,inplace=True)


In [5]:
scores=[]
importances=[]
print("Cross validation round: ")
for count in range(20):
    stdout.write("#"+str(count)+",")
    
    #rf=GradientBoostingClassifier(n_estimators=300)
    #rf=XGBClassifier(n_estimators=250,max_depth=6)
    rf=RandomForestClassifier(n_estimators=350,max_depth=6)
    
    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.05)
    
    # Augmenting minority class
    minoritySamp=np.logical_and(ytrain==1,np.random.rand(len(ytrain))<.85)
    xtrain=np.concatenate((xtrain,xtrain[minoritySamp]),axis=0)
    ytrain=np.concatenate((ytrain,np.ones(np.sum(minoritySamp))),axis=0)
    
    rf.fit(xtrain,ytrain)
    importances.append(rf.feature_importances_)
    scores.append([accuracy_score(ytest,rf.predict(xtest)),precision_score(ytest,rf.predict(xtest)),recall_score(ytest,rf.predict(xtest))])

print()
scores=pd.DataFrame(scores,columns=['Accuracy','Precision','Recall'])
print(np.mean(scores))

Cross validation round: 
#0,#1,#2,#3,#4,#5,#6,#7,#8,#9,#10,#11,#12,#13,#14,#15,#16,#17,#18,#19,
Accuracy     0.724048
Precision    0.568122
Recall       0.675782
dtype: float64
